## Session 3 - Train a REGRESSION Model

## SETUP


First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', size=12) 
plt.rc('figure', figsize = (12, 5))

# Settings for the visualizations
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

import pandas as pd
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Linear Regression

Linear regression is a simple technique that is useful for predicted problems.

linear regression pros:
* widely used
+ runs fast
+ easy to use (not a lot of tuning required)
+ highly interpretable
+ basis for many other methods

## Linear regression using the Normal Equation

In [ ]:
x = np.array([2.2, 4.3, 5.1, 5.8, 6.4, 8.0])
y = np.array([0.4, 10.1, 14.0, 10.9, 15.4, 18.5])
plt.plot(x,y,"o")
plt.xlabel("$x_1 $", fontsize=18)
plt.ylabel("$y $", rotation=90, fontsize=18)
plt.show()

## Ordinary Least Squares

$$\textbf{y} = b_0+b_1 \textbf{x}$$

Ordinary Least Squares (OLS) is the simplest and most common **estimator** in which the two $b$'s are chosen to minimize the sum of squared distance between the predicted values and the actual values. 

Given the set of samples $(\textbf{x},\textbf{y})$, the objective is to minimize:

$$ ||b_0 + b_1 \textbf{x} -  \textbf{y} ||^2_2 = \sum_{j=1}^n (b_0+b_1 x_{j} -  y_j )^2,$$ with respect to $b_0, b_1$.

This expression is often called **sum of squared errors of prediction (SSE)**.

## How to compute the OLS: Scipy.optimize

In [ ]:
# To understand the use of zip in the next code:
list(zip([2,3,4,5,6],[40,50,60,70,80]))

In [ ]:
from scipy.optimize import fmin

# Minimize the sum of squares using a lambda function

sse = lambda b, x, y: np.sum((y - b[0] - b[1]*x) ** 2) # Store the sum of squared differences function
# Lambda function is a small anonymous function. 
# It can take any number of arguments, but can only have one expression. 
# Syntax "lambda arguments : expression"

b0,b1 = fmin(sse, [0,1], args=(x,y)); # Minimize the sum of squared differences
# [0,1] is the initial guess for w[0] and w[1] in function sse.

plt.plot(x, y, 'ro')
plt.plot([0,10], [b0, b0+b1*10], alpha=0.8) # Add the regression line, colored in blue
for xi, yi in zip(x,y):
    plt.plot([xi]*2, [yi, b0+b1*xi], "k:") # Add pointed black line to illustrate the errors
plt.xlim(2, 9); plt.ylim(0, 20) # Restrict the domain
plt.show()

We can minimize other criteria, such as the **sum of absolute differences between the predicted values and the actual values**. 

In [ ]:
sabs = lambda b, x, y: np.sum(np.abs(y - b[0] - b[1]*x)) # Lambda function 

b0,b1 = fmin(sabs, [0,1], args=(x,y)) # Minimize the sum of absolute differences
plt.plot(x, y, 'ro')
plt.plot([0,10], [b0, b0+b1*10]) # Add the regression line, colored in blue
for xi, yi in zip(x,y):
    plt.plot([xi]*2, [yi, b0+b1*xi], "k:") # Add pointed black line to illustrate the errors
plt.xlim(2, 9); plt.ylim(0, 20) # Restrict the domain

### Example 1: Adversiting dataset

Let's play with an Adversiting dataset from the book "Introduction to Statistical Learning"

**The Data** 

A data frame with 200 observations on the following 4 variables (TV ,Radio, Newspaper, Sales)

OLS is a popular approach for several reasons. 

+ It is computationally cheap to calculate the coefficients. 
+ It is easier to interpret than more sophisticated models. In situations where the goal is understanding a simple model in detail, rather than estimating the response well, they can provide insight into what the model captures. 
+ Finally, in situations where there is a lot of noise, it may be hard to find the true functional form, so a constrained model can perform quite well compared to a complex model which is more affected by noise.

The resulting model is represented as follows:

$$\widehat{\textbf{y}} = \widehat{b}_0+\widehat{b}_1 \textbf{x}$$

Here the hats on the variables represent the fact that they are estimated from the data we have available.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/scikit-learn-videos/master/data/Advertising.csv',index_col=0)
df.head()

In [ ]:
print(df.shape)

What are the features?

* TV: advertising dollars spent on TV for a single product in a given market (in thousands of dollars)
* Radio: advertising dollars spent on Radio
* Newspaper: advertising dollars spent on Newspaper

What is the response?

* Sales: sales of a single product in a given market (in thousands of items)

What else do we know?

Because the response variable is continuous, this is a regression problem.
There are 200 observations (represented by the rows), and each observation is a single market

In [ ]:
X = df[['TV']]
y = df[['Sales']]
plt.plot(X,y,"o", alpha=0.3)
plt.xlabel("$x_1 : TV$", fontsize=18)
plt.ylabel("$y : Sales$", rotation=90, fontsize=18)
plt.show()

In [ ]:
X_b = np.c_[np.ones((200, 1)), X]  # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

In [ ]:
theta_best

In [ ]:

X_new = np.array([[0], [300]])
X_new_b = np.c_[np.ones((2, 1)), X_new]  # add x0 = 1 to each instance
y_predict = X_new_b.dot(theta_best)
y_predict

The figure in the book actually corresponds to the following code, with a legend and axis labels:

In [ ]:
plt.plot(X_new, y_predict, "r-", linewidth=2, label="Predictions")
plt.plot(X, y, "o",alpha=0.3)
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper left", fontsize=14)
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
lin_reg.predict(X_new)

As you can see, in this case, far values are penalized less.

## Performing Linear Regression using Statsmodel package

In [ ]:
import statsmodels.formula.api as smf

import statsmodels.api as sm
from scipy import stats
model = smf.ols(formula='Sales ~ TV', data=df)
model = model.fit()
print(model.summary())

In [ ]:

y_pred = model.predict(X)
plt.scatter(X, y, alpha=0.3, color='orchid')
plt.plot(X, y_pred, '-', color='darkorchid', linewidth=2)
plt.show()

## Performing Linear Regression using SciKitLearn package

In [ ]:
from sklearn import linear_model

from sklearn.metrics import mean_squared_error, r2_score


lin_reg = linear_model.LinearRegression()
lin_reg.fit(X, y)

y_pred = lin_reg.predict(X)
# plot the output

# Plot outputs
plt.scatter(X, y, alpha=0.3, color='orchid')
plt.plot(X, y_pred, '-', color='darkorchid', linewidth=2)
plt.show()


# The coefficients
print('Coefficients: \n', lin_reg.coef_)
# The mean squared error
print('Mean squared error: %.3f'
      % mean_squared_error(y, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination (R^2): %.3f'
      % r2_score(y, y_pred))

## Multiple Linear Regression
Simple linear regression can easily be extended to include multiple features. This is called multiple linear regression:

$$ Y = \beta_0 + \beta_1X_1 + ... \beta_pX_p + \epsilon $$

We interpret $\beta_j$ as the average effect on $Y$ of a one unit increase in $X_j$ , holding all other predictors fixed. In the advertising example, the model becomes:

$$ {\color{red}{sales}} = \beta_0 + \beta_1 ~x~ {\color{red}{TV}} + \beta_2 ~x~ {\color{red}{radio}} + \beta_3 ~x~ {\color{red}{newspaper}} + \epsilon $$

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.pairplot(df, x_vars=['TV','Radio','Newspaper'],y_vars='Sales', height=7, aspect=0.7)
plt.show()

In [ ]:
## with statsmodel
model = smf.ols(formula='Sales ~ TV + Radio', data=df)
model = model.fit()
print(model.summary())

In [ ]:
features = ['TV','Radio','Newspaper']
lin_reg = LinearRegression()
lin_reg.fit(df[features], y)

y_pred = lin_reg.predict(df[features])

# The coefficients
print('Coefficients: \n', lin_reg.coef_)
# The mean squared error
print('Mean squared error: %.3f'
      % mean_squared_error(y, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination (R^2): %.3f'
      % r2_score(y, y_pred))

We can see how the Coefficient of determination (R^2) has improved from 0.612 to 0.897

In [ ]:
corrMatrix = df.corr()
print (corrMatrix)

### Example 2:  Macroeconomic dataset

To start with we load the Longley dataset of US macroeconomic data from the R datasets website.

In [ ]:
import pandas as pd
# Read data
df = pd.read_csv('http://vincentarelbundock.github.io/Rdatasets/csv/datasets/longley.csv', index_col=0)
df.columns = ['GNPdeflator', 'GNP', 'Unemployed', 'ArmedForces', 'Population','Year', 'Employed']
df.head()

In [ ]:
print(df.shape)

Macroeconomic data from 1947 to 1962.

We will use the variable Total Derived Employment ('Employed') as our response $\textbf{y}$ and Gross National Product ('GNP') as our predictor $\textbf{x}$.

We also add a constant term so that we fit the intercept of our linear model: $X=(\textbf{1},\textbf{x})$

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.pairplot(df, x_vars=['GNPdeflator','Unemployed','ArmedForces','Population','Year','Employed'],y_vars='GNP', height=7, aspect=0.7)
plt.show()

In [ ]:
# another interesting graph from seaborn
sns.pairplot(df)
plt.show()

Let's create a model

In [ ]:
## with sciklearn
features = ['GNPdeflator','Unemployed','ArmedForces','Population','Year','Employed']
target = ['GNP']

X = df[features]
y = df[target]

lin_reg = LinearRegression()
lin_reg.fit(X, y)

y_pred = lin_reg.predict(X)

# The coefficients
print('Coefficients: \n', lin_reg.coef_)
# The mean squared error
print('Mean squared error: %.3f'
      % mean_squared_error(y, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination (R^2): %.3f'
      % r2_score(y, y_pred))

In [ ]:
## with statsmodel
model = smf.ols(formula='GNP ~ GNPdeflator + Unemployed + ArmedForces + Population + Year + Employed', data=df)
model = model.fit()
print(model.summary())

We can see signs of non linearity in the data which has not been captured by the model. 

In order to capture this non-linear effects, we have another type of regression known as polynomial regression. See below.

## Regularized Models

### Ridge Regression.
Ridge Regression penalizes the coefficients if they are too far from zero, thus enforcing them to be small in a continuous way. This way, it decreases model complexity while keeping all variables in the model.

For that, Ridge regression adds a **$\ell_2$-norm** regularization term to the sum of squared errors of prediction (SSE). Given the set of samples  (𝑋,𝐲) , the objetive is to minimize:

$$ minimize(\sum_{i=0}^n (y_i - B_0- \sum_{j=1}^pB_jx_{ij})^2 - \lambda\sum_{j=1}^pB_j^2) $$

### Lasso Regression:

Often, in real problems, there are uninformative variables in the data which prevent proper modeling of the problem and thus, the building of a correct regression model. In such cases, a feature selection process is crucial to select only the informative features and discard non-informative ones. This can be achieved by sparse methods which use a penalization approach, such as **Lasso** (least absolute shrinkage and selection operator) to set some model coefficients to zero (thereby discarding those variables). Sparsity can be seen as an application of Occam’s razor: prefer simpler models to complex ones.
For that, Lasso regression adds a **$\ell_1$-norm** regularization term to the sum of squared errors of prediction (SSE).  Given the set of samples  (𝑋,𝐲) , the objetive is to minimize:

$$ minimize(\sum_{i=0}^n (y_i - B_0- \sum_{j=1}^pB_jx_{ij})^2 - \lambda\sum_{j=1}^p|B_j|)$$

#### Geometric explanantion:
The left panel shows L1 regularization (lasso regularization) and the right panel L2 regularization (Ridge regression). The ellipses indicate the distribution for no regularization. The blue lines show the constraints due to regularization (limiting  𝜃2  for ridge regression and  |𝜃|  for Lasso regression). The corners of the L1 regularization create more opportunities for the solution to have zeros for some of the weights.

![alt text](https://miro.medium.com/max/1400/1*Jd03Hyt2bpEv1r7UijLlpg.png "Regularization")




In [ ]:
## Ridge Regression
regr_ridge = linear_model.Ridge(alpha=.3) # Create a Ridge regressor
regr_ridge.fit(X, y)  # Perform the fitting

print('Coeff and intercept: {} {}'.format(regr_ridge.coef_,  regr_ridge.intercept_))
coef = pd.Series(np.abs(regr_ridge.coef_[0]),features).sort_values()
coef.plot(kind='bar', title='Ridge Coefficients',ylabel="|$b_j$|")

In [ ]:
## Lasso Regression
regr_lasso = linear_model.Lasso(alpha=.3,tol=0.001) # Create a Ridge regressor
regr_lasso.fit(X, y)  # Perform the fitting

print('Coeff and intercept: {} {}'.format(regr_lasso.coef_,  regr_lasso.intercept_))

coef = pd.Series(np.abs(regr_lasso.coef_),features).sort_values()
coef.plot(kind='bar', title='Lasso Coefficients',ylabel="|$b_j$|")

In [ ]:
n_alphas = 100
alphas = np.logspace(-2, 5, n_alphas)

coefs_ridge = []

for l in alphas:
    regr_ridge = linear_model.Ridge(alpha=l) # Create a Ridge regressor
    regr_ridge.fit(X, y)  # Perform the fitting
    coefs_ridge.append(regr_ridge.coef_[0])
    
coefs_lasso = []
for l in alphas:
    regr_lasso = linear_model.Lasso(alpha=l,tol =0.001) # Create a Ridge regressor
    regr_lasso.fit(X, y)  # Perform the fitting
    coefs_lasso.append(regr_lasso.coef_)
# #############################################################################
# Display results

fig, axs = plt.subplots(1, 2, figsize=(20, 6), sharey=True)


axs[0].plot(alphas, coefs_ridge)
axs[0].set_xscale('log')
axs[0].set_title('Ridge coefficients as a function of the regularization')
axs[0].axis('tight')
axs[0].set_xlabel('alpha')
axs[0].set_ylabel('weights')

axs[1].plot(alphas, coefs_lasso)
axs[1].set_xscale('log')
axs[1].set_title('Lasso coefficients as a function of the regularization')
axs[1].axis('tight')
axs[1].set_xlabel('alpha')
axs[1].set_ylabel('weights')
plt.show()


### EXERCICE: Create and evalualuate a regression model for **THIS** dataset

Remember the ML pipeline: 

#### ML Pipeline
* Setting up the environment and data import
* Understanding the data
* Exploratory Data Analysis
* Linear Regression Model
* Preparation and splitting the data
* Train and Test the Model
* Train and Test New Model
* Compare the models
* Model Performance
* Applying on new data

### STEP 1 - Setting up the enviroment and data import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
df = pd.read_csv('https://raw.githubusercontent.com/ssegui/ml_ub/master/notebooks/dataset/regression_healthcare/datasets_13720_18513_insurance.csv')
df['charges'] = np.log(df['charges'])
df.shape

In [ ]:
df.head()

### Step 2 - Understanding the data

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# AGE: Insurance contractor age, year. 
#       min age : 18
#       max age : 64

sns.distplot(df.age,bins=65)
plt.show()

In [ ]:
# SEX: Insurance contractor gender, [female, male ]
sns.countplot(data=df, y = 'sex')
plt.show()

In [ ]:
# BMI: Body mass index, providing an understanding of body, weights that are relatively 
# high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of 
# height to weight, ideally 18.5 to 24.9
sns.distplot(df.bmi,bins=65)
plt.show()


In [ ]:
#Smoker: smoking, [yes, no]
sns.countplot(data=df,y='smoker')
plt.show()

In [ ]:
sns.catplot(x="smoker", kind="count",hue = 'sex', palette="pink", data=df)
plt.show()

In [ ]:
# Children: number of children covered by health insurance / Number of dependents
sns.countplot(data=df,y='children')
plt.show()

In [ ]:
# Children: number of children covered by health insurance / Number of dependents
sns.countplot(data=df,y='region')
plt.show()

In [ ]:
# Charges: Individual medical costs billed by health insurance, $ #predicted value
sns.distplot(df.charges)
plt.show()

sns.boxplot(x=df["charges"])
plt.show()

### Divide data into train and test set

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.8, random_state=42)

In [ ]:
test_set.head()

## Prepare the data for Machine Learning algorithms


In [ ]:
from sklearn.impute import SimpleImputer
cat_attribs = ['sex','region','smoker']
num_attribs = ['bmi','age','children']

X_num = df[num_attribs]
X_cat = df[cat_attribs]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])
X_train = full_pipeline.fit_transform(train_set)
y_train = train_set['charges']
X_train.shape


tmp = pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train.values)], axis=1)
tmp.columns = ['bmi','age','children','sex_0','sex_1','region_0','region_1','region_2','region_3','smoker_0','smoker_1','charges']

corrMatrix = tmp.corr()
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(corrMatrix, vmax=.8, square=True, annot=True,ax=ax)
plt.show()

## Select and train a model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

In [ ]:
# let's try the full preprocessing pipeline on a few training instances
some_data = train_set.iloc[:5]
some_labels = train_set[['charges']].iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predicted Charges:", lin_reg.predict(some_data_prepared))

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


y_pred = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, y_pred)
lin_rmse = np.sqrt(lin_mse)
lin_mae = mean_absolute_error(y_train, y_pred)

print(lin_rmse,lin_mae)

### Fine-tune your model


In [ ]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())


In [ ]:
from sklearn.ensemble import RandomForestRegressor


def cross_val_evaluation(model,X_train,y_train,model_name):
    scores = cross_val_score(model, X_train, y_train,cv=5)
    print("\n ",model_name)
    display_scores(scores)

lin_reg = LinearRegression()
cross_val_evaluation(lin_reg,X_train,y_train,'Linear Regression')

ridge_reg = linear_model.Ridge(alpha=.3) # Create a Ridge regressor
cross_val_evaluation(ridge_reg,X_train,y_train,'Ridge Regression')

lasso_reg = linear_model.Lasso(alpha=0.01) # Create a Ridge regressor
cross_val_evaluation(lasso_reg,X_train,y_train,'Lasso Regression')

forest_reg = RandomForestRegressor(random_state=42)
cross_val_evaluation(forest_reg,X_train,y_train,'Random Forest')

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'alpha': np.logspace(-2, 5, 100)}]

ridge_reg = linear_model.Ridge() # Create a Ridge regressor
grid_search_ridge = GridSearchCV(ridge_reg, param_grid, cv=5,scoring="neg_mean_squared_error",
                                 return_train_score=True, n_jobs=-1)

grid_search_ridge.fit(X_train, y_train)
print(grid_search_ridge.best_params_)
print(grid_search_ridge.best_score_)
pd.DataFrame(grid_search_ridge.cv_results_)

In [ ]:
lasso_reg = linear_model.Lasso(tol=0.01) # Create a Ridge regressor
grid_search_lasso = GridSearchCV(lasso_reg, param_grid, cv=5,scoring="neg_root_mean_squared_error",
                                 return_train_score=True,n_jobs=-1)
grid_search_lasso.fit(X_train, y_train)
print(grid_search_lasso.best_params_)
print(grid_search_lasso.best_score_)

pd.DataFrame(grid_search_lasso.cv_results_)

In [ ]:
from sklearn.model_selection import validation_curve
param_range= np.logspace(-2, 2, 20)

train_scores, valid_scores = validation_curve(ridge_reg, X_train, y_train, param_name= "alpha",
                                              param_range= param_range,
                                              cv=3)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(valid_scores, axis=1)
test_scores_std = np.std(valid_scores, axis=1)


plt.title("Validation Curve with Ridge Regression")
plt.xlabel(r"$\alpha$")
plt.ylabel("Score")

lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

In [ ]:
# The same with lasso model
param_range= np.logspace(-2, 3, 20)

train_scores, valid_scores = validation_curve(lasso_reg, X_train, y_train, param_name= "alpha",
                                              param_range= param_range,
                                              cv=3)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(valid_scores, axis=1)
test_scores_std = np.std(valid_scores, axis=1)


plt.title("Validation Curve with Ridge Regression")
plt.xlabel(r"$\alpha$")
plt.ylabel("Score")

lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

### Evaluate test set

In [ ]:
# Check for Linearity
X_test = full_pipeline.transform(test_set)
y_test = test_set['charges']

lin_reg.fit(X_train,y_train)
y_pred = lin_reg.predict(X_test)

plt.figure(figsize=(14,5))
sns.scatterplot(y_test,y_pred,color='g')
plt.title('Check for Linearity:\n Actual Vs Predicted value')
plt.ylabel('Predicted')
plt.xlabel('Real')

plt.show()

In [ ]:
# Check for Linearity --  LASSSO
X_test = full_pipeline.transform(test_set)
y_test = test_set['charges']

grid_search_lasso.fit(X_train,y_train)
y_pred = grid_search_lasso.predict(X_test)

plt.figure(figsize=(14,5))
sns.scatterplot(y_test,y_pred,color='g')
plt.title('Check for Linearity:\n Actual Vs Predicted value')
plt.ylabel('Predicted')
plt.xlabel('Real')

plt.show()

In [ ]:
# Check for Linearity --  RIGE
X_test = full_pipeline.transform(test_set)
y_test = test_set['charges']

grid_search_lasso.fit(X_train,y_train)
y_pred_lasso = grid_search_lasso.predict(X_test)

grid_search_ridge.fit(X_train,y_train)
y_pred_ridge = grid_search_ridge.predict(X_test)


plt.figure(figsize=(14,5))
sns.scatterplot(y_test,y_pred_ridge,color='g',alpha=0.4)
sns.scatterplot(y_test,y_pred_lasso,color='r',alpha=0.4)

plt.title('Check for Linearity:\n Actual Vs Predicted value')
plt.ylabel('Predicted')
plt.xlabel('Real')

plt.show()

In [ ]:
print("LINEAR REGRESSION")
y_pred = lin_reg.predict(X_test)
# The mean squared error
print(' Mean squared error: %.3f'%  np.sqrt(mean_squared_error(y_test, y_pred)))
# The coefficient of determination: 1 is perfect prediction
print(' Coefficient of determination (R^2): %.3f'% r2_score(y_test, y_pred))

print("\nRIDGE REGRESSION")
print(grid_search_ridge.best_params_)
y_pred = grid_search_ridge.predict(X_test)
# The mean squared error
print(' Mean squared error: %.3f'%  np.sqrt(mean_squared_error(y_test, y_pred)))
# The coefficient of determination: 1 is perfect prediction
print(' Coefficient of determination (R^2): %.3f'% r2_score(y_test, y_pred))

print("\nLASSO REGRESSION")
print(grid_search_lasso.best_params_)
y_pred = grid_search_lasso.predict(X_test)
# The mean squared error
print(' Mean squared error: %.3f'%  np.sqrt(mean_squared_error(y_test, y_pred)))
# The coefficient of determination: 1 is perfect prediction
print(' Coefficient of determination (R^2): %.3f'% r2_score(y_test, y_pred))

### EXERCICE: Create and evalualuate a regression model for **THIS** dataset

Remember the ML pipeline: 

#### ML Pipeline
* Setting up the environment and data import
* Understanding the data
* Exploratory Data Analysis
* Linear Regression Model
* Preparation and splitting the data
* Train and Test the Model
* Train and Test New Model
* Compare the models
* Model Performance
* Applying on new data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
housing = pd.read_csv('./dataset/housing-snapshot/train_set.csv',index_col=0)
#housing['charges'] = np.log(housing['Price'])
housing.shape

In [ ]:
housing.head(10)

In [ ]:
housing.info()

In [ ]:
housing = housing.fillna(housing.mean())
housing = housing.dropna()

In [ ]:
housing_num = housing.select_dtypes(include=[np.number])
numerical_features = list(housing_num)
categorical_features = list(housing.select_dtypes(include=[object]))
print(categorical_features)

In [ ]:
housing = housing.drop(['Address','Regionname','Postcode','SellerG','Date'],axis=1)
housing = pd.get_dummies(housing,columns= ['Type','Method','CouncilArea','Suburb'],drop_first=True)

In [ ]:
housing = housing.fillna(housing.mean())

In [ ]:
X_train = housing.drop("Price", axis=1) # drop labels for training set
y_train = housing["Price"].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X_train.values)
X = pd.DataFrame(X)
X.columns = X_train.columns.tolist()
X_train = X

In [ ]:
X_train

# Ridge & Lasso

In [ ]:
## Ridge Regression
regr_ridge = linear_model.Ridge(alpha=.3) # Create a Ridge regressor
regr_ridge.fit(X_train, y_train)  # Perform the fitting

In [ ]:
def pretty_print_coefs(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)

In [ ]:
print ("Ridge model:", pretty_print_coefs(regr_ridge.coef_))

In [ ]:
## Lasso Regression
regr_lasso = linear_model.Lasso(alpha=.3,tol=0.001) # Create a Ridge regressor
regr_lasso.fit(X_train, y_train)  # Perform the fitting


In [ ]:
print ("Lasso model:", pretty_print_coefs(regr_lasso.coef_))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import RidgeCV

# load data
X = X_train
y = y_train

# Use L2 penalty
estimator = RidgeCV(cv=5, normalize = True)

# Set a minimum threshold of 0.25
sfm = SelectFromModel(estimator, threshold=0.25, prefit=False, norm_order=2, max_features=12)

sfm.fit(X, y)

feature_idx = sfm.get_support()
feature_name = X.columns[feature_idx]
feature_name

# n_features = sfm.transform(X).shape[1]
# n_features

In [ ]:
print ("Ridge model:", pretty_print_coefs(regr_lasso.coef_))

In [ ]:
ridge_features = ['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'BuildingArea',
       'YearBuilt', 'Lattitude', 'Longtitude', 'Type_u', 'CouncilArea_Bayside',
       'CouncilArea_Boroondara']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

# load data
X = X_train
y = y_train

# Use L1 penalty
estimator = LassoCV(cv=5, normalize = True)

# Set a minimum threshold of 0.25
sfm = SelectFromModel(estimator, threshold=0.25, prefit=False, norm_order=1, max_features=12)

sfm.fit(X, y)

feature_idx = sfm.get_support()
feature_name = X.columns[feature_idx]
feature_name

# n_features = sfm.transform(X).shape[1]
# n_features

In [ ]:
lasso_features = ['Rooms', 'Distance', 'Bathroom', 'Car', 'YearBuilt', 'Lattitude',
       'Type_t', 'Type_u', 'CouncilArea_Boroondara', 'CouncilArea_Brimbank',
       'CouncilArea_Wyndham', 'Suburb_Brighton']

## Back Foward

In [ ]:
def backward_regression(X, y,initial_list=[],threshold_in=0.01,threshold_out = 0.05,verbose=False):
    included=list(X.columns)
    while True:
        changed=False
        model = sm.OLS(list(y), sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop  with p-value '.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

ols_features = backward_regression(X_train, y_train)

In [ ]:
ols_features = ols_features[0:12]
ols_features

In [ ]:
X_train_ols = X_train[ols_features]
X_train_ols

In [ ]:
X_train_ridge = X_train[ridge_features]
X_train_ridge

In [ ]:
X_train_lasso = X_train[lasso_features]
X_train_lasso

# TEST OLS

In [ ]:
X_test = pd.read_csv('./dataset/housing-snapshot/test_set.csv',index_col=0)
X_test.shape

In [ ]:
X_test = X_test.fillna(X_test.mean())
X_test = X_test.drop(['Address','Regionname','Postcode','SellerG','Date'],axis=1)
X_test = pd.get_dummies(X_test,columns= ['Type','Method','CouncilArea','Suburb'],drop_first=True)

X_test.shape



In [ ]:
# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
print(missing_cols)
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]
X_test.shape

In [ ]:
X = sc_X.transform(X_test.values)
X = pd.DataFrame(X)
X.columns = X_test.columns.tolist()
X_test = X

# Select best Model

In [ ]:
X_test_ols = X_test[ols_features]
X_test_ridge = X_test[ridge_features]
X_test_lasso = X_test[lasso_features]

# Lineal Regression

# MODELOS CON 12 features diferentes (SM,Ridge, Lasso)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train_ols, y_train)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
ols_predict = lin_reg.predict(X_test_ols)
ols_predict

In [ ]:
df_output = pd.DataFrame(ols_predict)
df_output = df_output.reset_index()
df_output.columns = ['index','Price']

df_output.to_csv('ols.csv',index=False)

In [ ]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
lin_reg = LinearRegression()
cross_val_evaluation(lin_reg,X_train_ols,y_train,'Linear Regression')

# MODELO RIDGE

In [ ]:
ridge_reg = linear_model.Ridge(alpha=.3,normalize = True) # Create a Ridge regressor
cross_val_evaluation(ridge_reg,X_train_ridge,y_train,'Ridge Regression')

# MODELO LASSO

In [ ]:
lasso_reg = linear_model.Lasso(alpha=.3, normalize = True) # Create a Ridge regressor
cross_val_evaluation(lasso_reg,X_train_lasso,y_train,'Lasso Regression')

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'alpha': np.logspace(-2, 5, 100)}]

ridge_reg = linear_model.Ridge(normalize = True) # Create a Ridge regressor
grid_search_ridge = GridSearchCV(ridge_reg, param_grid, cv=5,scoring="neg_mean_squared_error",
                                 return_train_score=True, n_jobs=-1)

grid_search_ridge.fit(X_train_ridge, y_train)
print('Best Score: ', grid_search_ridge.best_score_)
print('Best Params: ', grid_search_ridge.best_params_)
pd.DataFrame(grid_search_ridge.cv_results_)

In [ ]:
from sklearn.model_selection import validation_curve
param_range= np.logspace(-2, 5, 20)

train_scores, valid_scores = validation_curve(ridge_reg, X_train_ridge, y_train, param_name= "alpha",
                                              param_range= param_range,
                                              cv=3)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(valid_scores, axis=1)
test_scores_std = np.std(valid_scores, axis=1)


plt.title("Validation Curve with Ridge Regression")
plt.xlabel(r"$\alpha$")
plt.ylabel("Score")

lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

In [ ]:
ridge_reg = linear_model.Ridge(alpha=0.01,normalize = True)
ridge_reg.fit(X_train_ridge, y_train)
ridge_reg = ridge_reg.predict(X_test_ridge)


df_output = pd.DataFrame(ridge_reg)
df_output = df_output.reset_index()
df_output.columns = ['index','Price']
df_output.to_csv('ridge.csv',index=False)

In [ ]:
lasso_reg = linear_model.Lasso(tol=0.01, normalize = True) # Create a Ridge regressor
param_grid = [{'alpha': np.logspace(-2, 5, 1000)}]

grid_search_lasso = GridSearchCV(lasso_reg, param_grid, cv=5,scoring="neg_root_mean_squared_error",
                                 return_train_score=True, n_jobs=-1)
grid_search_lasso.fit(X_train_lasso, y_train)
print('Best Score: ', grid_search_lasso.best_score_)
print('Best Params: ', grid_search_lasso.best_params_)

pd.DataFrame(grid_search_lasso.cv_results_)

In [ ]:
# The same with lasso model
param_range= np.logspace(-2, 5, 20)

train_scores, valid_scores = validation_curve(lasso_reg, X_train_lasso, y_train, param_name= "alpha",
                                              param_range= param_range,
                                              cv=3)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(valid_scores, axis=1)
test_scores_std = np.std(valid_scores, axis=1)


plt.title("Validation Curve with Ridge Regression")
plt.xlabel(r"$\alpha$")
plt.ylabel("Score")

lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

In [ ]:
lasso_reg = linear_model.Lasso(tol=0.01,alpha=5.317,normalize = True)
lasso_reg.fit(X_train_lasso, y_train)
lasso_reg = lasso_reg.predict(X_test_lasso)

df_output = pd.DataFrame(lasso_reg)
df_output = df_output.reset_index()
df_output.columns = ['index','Price']
df_output.to_csv('laso.csv',index=False)


In [ ]:
## QUESTION: What happens if in the problem of House Prediction Price instead of estimating the price, we estimate the log of the price?